<a href="https://colab.research.google.com/github/aijelly/I-want-to-study-Data-Science/blob/master/n214a_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 1 / NOTE 4*

# 📝 Assignment
---
# Logistic Regression

이번에는 [**Cardiovascular Disease dataset**](https://www.kaggle.com/sulianova/cardiovascular-disease-dataset) 데이터셋을 사용해 봅시다. 데이터에 대한 설명을 캐글 페이지에서 잘 살펴보시고 과제를 진행해 주세요!

### 1) 데이터를 전처리 하겠습니다.
- 데이터를 불러오세요.(read_csv 사용)
- missing value를 확인하세요.
- 중복된 샘플이 있는지 확인하세요.
  - id가 동일하면 중복 샘플입니다. id로 중복 여부를 확인하세요.
- 이후 id 칼럼은 drop하세요.
- **중복된 샘플을 제거하고 남은 샘플의 수를 제출하세요.**

In [162]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/cardio/cardio_train.csv', sep=';')

In [163]:
df.describe()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
count,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,49972.419900,19468.865814,1.349571,164.359229,74.205690,128.817286,96.630414,1.366871,1.226457,0.088129,0.053771,0.803729,0.499700
std,28851.302323,2467.251667,0.476838,8.210126,14.395757,154.011419,188.472530,0.680250,0.572270,0.283484,0.225568,0.397179,0.500003
min,0.000000,10798.000000,1.000000,55.000000,10.000000,-150.000000,-70.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,25006.750000,17664.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,50001.500000,19703.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000
75%,74889.250000,21327.000000,2.000000,170.000000,82.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,99999.000000,23713.000000,2.000000,250.000000,200.000000,16020.000000,11000.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000


In [164]:
df = df.copy()
df.isna()
df = df.drop_duplicates()
df = df.drop('id',axis=1)

### 2) 다운로드 한 훈련 데이터를 훈련/검증/테스트 세트로 나누어 주세요.
- 타겟 특성을 확인하고 타겟 변수를 나누어 줍니다.
- 훈련과 테스트 데이터로 나누어 줍니다(train_test_split 사용, train/test = 80/20, random_state=2)
- 훈련 데이터를 훈련과 검증 데이터로 나누어 줍니다(train_test_split 사용, train/validation = 80/20)
- **검증 데이터(X_val)의 개수를 제출하세요.**

In [165]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, train_size=0.8, test_size=0.2, random_state=2)

In [166]:
train, val = train_test_split(train, train_size=0.8, test_size=0.2, random_state=2)
print("train shape: ", train.shape)
print("test shape: ", test.shape)
print("val shape: ", val.shape)
# train_test_split()

train shape:  (44800, 12)
test shape:  (14000, 12)
val shape:  (11200, 12)


### 3) 기준모델을 설정하세요.
- 우리가 예측하고자 하는 column은 `cardio`(Presence or absence of cardiovascular disease)입니다.
- 모델을 만들기 전에 최빈 클래스로 기준모델을 설정합니다.
- **훈련 데이터에 대하여 기준모델 정확도를 제출하세요.**

In [167]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [168]:
target = 'cardio'
X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]
X_test = test.drop(columns=target)
y_test = test[target]

In [169]:
X_train

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
7808,20396,1,157,60.0,120,80,3,3,0,0,1
35103,18174,2,177,64.0,110,60,1,1,1,0,0
383,19517,1,147,41.0,80,50,1,1,0,0,1
68839,18874,1,161,54.0,110,70,1,1,0,0,0
15529,18934,1,160,61.0,150,100,2,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
18506,22477,1,156,58.0,110,70,2,1,0,0,0
5313,15344,2,175,82.0,110,60,1,1,0,0,1
61096,16673,2,165,65.0,120,80,1,1,0,0,0
38973,20137,1,160,87.0,140,90,1,1,0,0,0


In [170]:
# mode(): Return the highest frequency value in a Series.
major = y_train.mode()[0]

# 타겟 샘플 수 만큼 0이 담긴 리스트를 만듭니다. 기준모델로 예측
y_pred = [major] * len(y_train)

In [171]:
from sklearn.metrics import accuracy_score
print("training accuracy: ", accuracy_score(y_train, y_pred))

training accuracy:  0.5008705357142857


### 4) 수치형 특성 값들을 표준화 하세요.
- `StandardScaler()`를 사용해서 수치형 특성만 표준화 하세요.

In [172]:
features = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']

In [173]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train.loc[:, features] = scaler.fit_transform(X_train.loc[:,features])
X_val.loc[:, features] = scaler.transform(X_val.loc[:, features])
X_test.loc[:, features] = scaler.transform(X_test.loc[:, features])

### 5) 데이터에서 범주형 성질을 지닌 특성을 찾아 원핫인코딩 하세요.
- `category_encoders`를 사용해서 `gender` 특성만 원핫인코딩 하세요. 
  - `use_cat_names=True`로 설정합니다. 
  - `cols`를 사용하여 인코딩을 적용할 특성을 설정할 수 있습니다.

In [175]:
!pip install category_encoders 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [176]:
from category_encoders import OneHotEncoder
encoder = OneHotEncoder(use_cat_names=True, cols=['gender'])
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)
X_val_encoded.head(3) 

,age,gender_1.0,gender_2.0,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
20630,0.705552,0,1,0.321675,-0.221396,0.184783,0.014104,1,1,0,0,1
58472,0.445057,1,0,-1.739070,-0.916259,-0.054656,-0.087151,1,1,0,0,1
54488,-1.644992,1,0,0.079234,-0.846772,-0.114516,-0.087151,1,1,0,0,1


### 6) 모델 학습
- Logistic Regression 모델을 인코딩된 데이터로 학습합니다. 
  - `max_iter=1000`으로 설정합니다. 
- **검증세트 정확도를 제출하세요.**

In [177]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=2, max_iter=1000)
model.fit(X_train_encoded, y_train)

LogisticRegression(max_iter=1000, random_state=2)

In [178]:
y_val_pred = model.predict(X_val_encoded)
model.score(X_val_encoded, y_val)

0.72625

### 7) 학습한 모델을 검증세트에서 최적화 시킨 후, 테스트세트에서 딱 한 번 정확도를 확인해 보세요.
- **테스트 세트 정확도를 제출하세요. (0.712 이상)**

In [179]:
y_pred = model.predict(X_test_encoded)
model.score(X_test_encoded, y_test)

0.7241428571428571

## 🔥 도전과제

### 8) EDA를 추가적으로 수행하고 아래 질문에 대한 본인만의 근거와 수행결과를 리포트해 보시고 서로 논해 보세요.
- 새로운 특성을 만들 수 있다면, 어떤 특성공학을 해 볼 수 있을까요?
- 아웃라이어가 있는 특성이 있다면, 어떤 기준으로 제거할 수 있을까요?
- [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) 문서를 보면 여러가지 종류의 Scalar가 있는데, 각 Scalar는 각각 어떤 상황에 적용하면 좋을까요? 본 데이터에서는 어떤 scalar를 적용해볼 수 있을까요?
- 모델 성능을 높이기 위해 어떤 튜닝을 추가해 볼 수 있을까요?

In [180]:
### 이곳에서 과제를 진행해 주세요 ### 